In [5]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from tqdm import tqdm

from utils.preprocessing import get_texts
from utils.preprocessing import get_texts, stop_words
import random
from sklearn.metrics import confusion_matrix 

In [14]:
df_esg_score = pd.read_excel("data/esg_score.xlsx", sheet_name = "data")

In [15]:
# TODO: Modify here for different sectors and score_types
# set the sector and ESG type for the analysis
sector = "Financial Services"
# sector = "Energy"

score_type = "governanceScore"
# score_type = "environmentScore"
# score_type = "socialScore"

In [16]:
df_esg_score["sector"].unique()

array(['Consumer Cyclical', 'Energy', 'Industrials', 'Healthcare',
       'Basic Materials', 'Consumer Defensive', 'Utilities', 'Technology',
       'Financial Services', 'Communication Services', 'Real Estate', nan],
      dtype=object)

In [17]:
tickers = df_esg_score[df_esg_score["sector"] == sector]["Company"]
esgs = df_esg_score[df_esg_score["sector"] == sector][["Company", "socialScore", "governanceScore", "environmentScore"]]

In [18]:
len(tickers)

64

In [19]:
esgs

,Company,socialScore,governanceScore,environmentScore
34,WFC,16.15,15.64,1.31
66,USB,15.85,12.44,1.71
75,SIVB,12.92,14.11,2.11
93,JPM,16.35,10.55,1.11
99,AIG,10.81,14.71,1.87
...,...,...,...,...
371,WU,7.02,5.93,1.29
408,SPGI,5.77,5.37,1.57
413,MSCI,5.41,5.39,1.61
426,NDAQ,4.56,5.36,1.81


In [13]:
score

34     15.64
66     12.44
75     14.11
93     10.55
99     14.71
       ...  
371     5.93
408     5.37
413     5.39
426     5.36
428     5.76
Name: governanceScore, Length: 64, dtype: float64

In [20]:
score = esgs[score_type]

In [21]:
score

34     15.64
66     12.44
75     14.11
93     10.55
99     14.71
       ...  
371     5.93
408     5.37
413     5.39
426     5.36
428     5.76
Name: governanceScore, Length: 64, dtype: float64

In [96]:
alpha = 0.5
upper_score = np.quantile(score, 1 - alpha)
lower_score = np.quantile(score, alpha)

In [97]:
bad_companies = esgs[esgs[score_type] > upper_score]["Company"].values
good_companies = esgs[esgs[score_type] < lower_score]["Company"].values

In [98]:
#training set
train_bad=random.sample(list(bad_companies), int(len(bad_companies)*0.7))
train_good=random.sample(list(good_companies), int(len(good_companies)*0.7))

#validation set
validate_bad = [i for i in bad_companies if i not in train_bad]
validate_good = [i for i in good_companies if i not in train_good]

# validate_good = pd.DataFrame(validate_good) 
# validate_bad = pd.DataFrame(validate_bad) 

In [99]:
validation = pd.DataFrame({'good':validate_good, 'bad':validate_bad})

In [100]:
#save the list of good and bad companies
validation_path = os.path.join("data", "validation_data")
if not os.path.isdir(validation_path):
    os.mkdir(validation_path)

validation.to_csv("data/validation_data/{}_{}_{}.csv".format(sector[:8], score_type[:3], alpha))

In [58]:
print(len(train_good))

20


In [59]:
print(len(validate_good))

9


In [60]:
# esgs[esgs[score_type] > upper_score]

In [61]:
#training data bad companies scores
train_bad_scores=pd.DataFrame()
for i in train_bad:
    df_bad=esgs[esgs['Company'] == i]
    train_bad_scores=train_bad_scores.append(df_bad)

In [62]:
#training data good companies scores
train_good_scores=pd.DataFrame()

for i in train_good:
    df_good=esgs[esgs['Company'] == i]
    train_good_scores =train_good_scores.append(df_good)

In [63]:
# bad_companies_score = esgs[esgs[score_type] > upper_score][score_type].values
# good_companies_score = esgs[esgs[score_type] < lower_score][score_type].values

good_companies_score_training=train_good_scores[score_type]
bad_companies_score_training=train_bad_scores[score_type]

In [64]:
avg_bad = np.mean(bad_companies_score_training)
avg_good = np.mean(good_companies_score_training)
print(avg_bad, avg_good)

11.138499999999999 6.109999999999999


In [143]:
print(upper_score, lower_score)

10.23 10.23


In [144]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))
good_cik = []
bad_cik = []
for ticker in train_good:    
    try:
        # for a given ticker, find its cik number through th ticker library
        good_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        good_cik.append('')

for ticker in train_bad:    
    try:
        # for a given ticker, find its cik number through th ticker library
        bad_cik.append(ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:])
    except:
        # if could not find cik, give it a empty cik
        bad_cik.append('')

C:\Users\Mohammed Al Harmoudi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [145]:
ret_good = get_texts(good_cik, train_good)

0it [00:00, ?it/s]
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping CIK 0001281761



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.30s/it]

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping CIK 0001281761



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.86s/it]
1it [00:49, 49.05s/it]
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.78s/it]
21it [10:24, 34.71s/it]
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping CIK 0001403161



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.84s/it]

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping CIK 0001403161



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.51s/it]
22it [10:48, 29.49s/it]


In [146]:
ret_bad = get_texts(bad_cik, train_bad)

0it [00:00, ?it/s]
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping CIK 0000899051



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.57s/it]

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping CIK 0000899051



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.64s/it]
3it [00:59, 41.29s/it]
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

...


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.07s/it]
16it [03:20,  7.72s/it]
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping CIK 0000080661



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.94s/it]

  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

Scraping CIK 0000080661



100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:09<00:00,  9.01s/it]
22it [03:56, 10.73s/it]


In [147]:
good_docs = ret_good["docs"]
bad_docs = ret_bad["docs"]

In [148]:
# TODO: Modify here for different ngram range
n_min = 2
n_max = 3
cv = CountVectorizer(max_df=0.7, stop_words=stop_words, max_features=200, ngram_range=(n_min, n_max))
word_count_vector = cv.fit_transform(good_docs + bad_docs)

C:\Users\Mohammed Al Harmoudi\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['10'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [149]:
feature_names = cv.get_feature_names()

In [150]:
count_feature = word_count_vector.toarray().sum(axis=0)

In [151]:
feature_names = cv.get_feature_names()

In [152]:
d = {"word": [], "good_score": [], "bad_score": [], "good_score_all": []
    , "bad_score_all": [], "count": [], "good_nums": [], "bad_nums": []}

for feature_idx, word in enumerate(feature_names):
    good_sum = bad_sum = good_num = bad_num = 0

    for i, doc_set in enumerate(good_docs):
        if word in doc_set:
            good_num += 1
            good_sum += good_companies_score[i]
    for i, doc_set in enumerate(bad_docs):
        if word in doc_set:
            bad_num += 1
            bad_sum += bad_companies_score[i]
    
    # print("word: {}".format(word))
    d["word"].append(word) 
    
    if good_num:
        d["good_score"].append(good_sum / good_num)
    else:
        d["good_score"].append(0)
    if bad_num:
        d["bad_score"].append(bad_sum / bad_num)
    else:
        d["bad_score"].append(0)

    d["good_score_all"].append(good_sum / len(good_docs))
    d["bad_score_all"].append(bad_sum / len(bad_docs))

    d["count"].append(count_feature[feature_idx])
    d["good_nums"].append(good_num)
    d["bad_nums"].append(bad_num)


In [153]:
df = pd.DataFrame(data=d)


In [154]:
df.head(20)

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums
0,accident year,0.730000,7.668571,0.066364,2.440000,2329,2,7
1,accompanying note,1.002308,7.452308,0.592273,4.403636,847,13,13
2,acquisition cost,0.867273,7.795625,0.433636,5.669545,1249,11,16
3,adjusted earnings,1.016667,6.565000,0.138636,0.596818,918,3,2
4,adjusted operating,1.042857,5.356000,0.331818,1.217273,1454,7,5
5,adjustment expense,0.925000,7.531111,0.168182,3.080909,1133,4,9
6,aflac japan,0.000000,5.870000,0.000000,0.266818,732,0,1
7,allowance loan,0.835833,8.014615,0.455909,4.735909,951,12,13
8,amount except,0.616000,9.350000,0.140000,2.125000,779,5,5
9,available sale,0.760000,7.723077,0.345455,4.563636,871,10,13


In [155]:
df["diff"] = abs(df["good_nums"] - df["bad_nums"])

In [156]:
df = df.sort_values("diff", ascending=False)#.head(60)

In [157]:
df.head(20)

,word,good_score,bad_score,good_score_all,bad_score_all,count,good_nums,bad_nums,diff
86,insurance subsidiary,0.925000,7.593846,0.168182,4.487273,1246,4,13,9
130,net interest income,0.798667,7.905000,0.544545,2.155909,2101,15,6,9
98,line business,0.931000,7.474444,0.423182,6.115455,1069,10,18,8
135,net revenue,0.900625,7.540000,0.655000,2.741818,867,16,8,8
116,loss ratio,0.955000,7.850833,0.173636,4.282273,846,4,12,8
127,mortgage loan,0.791000,8.332222,0.359545,6.817273,4022,10,18,8
76,future policy,0.925000,7.480000,0.168182,4.080000,1214,4,12,8
150,policy benefit,0.925000,7.610833,0.168182,4.151364,1121,4,12,8
85,insurance product,0.880000,7.723125,0.360000,5.616818,1022,9,16,7
91,invested asset,0.816000,8.008333,0.185455,4.368182,856,5,12,7


In [158]:
goodvbad_path = os.path.join("data", "training_goodvbad")
if not os.path.isdir(goodvbad_path):
    os.mkdir(goodvbad_path)

In [159]:
df.round(2).to_csv("data/training_goodvbad/{}_{}_{}_n{}-{}.csv".format(sector[:8], score_type[:3], alpha, n_min, n_max))

## Validation

In [4]:
#trainig data scores

#Technology_envirnomental
data_tech = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\training_goodvbad\Technolo_env_0.5_n2-3.csv'

#consumer cyclical_social
data_consumer = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\training_goodvbad\Consumer_soc_0.5_n2-3.csv'

#financial services_governance
data_financials = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\training_goodvbad\Financia_gov_0.5_n2-3.csv'

In [5]:
df_tech=pd.read_csv(data_tech).iloc[:,1:]
df_consumer=pd.read_csv(data_consumer).iloc[:,1:]
df_financials=pd.read_csv(data_financials).iloc[:,1:]

In [6]:
df_tech.isGood.value_counts()

AttributeError: 'DataFrame' object has no attribute 'isGood'

AttributeError: 'DataFrame' object has no attribute 'isGood'

In [18]:
df_consumer.isGood.value_counts()

 1    91
 0    79
-1    30
Name: isGood, dtype: int64

In [7]:
#load validation companies

#consumer cyclical_social
validation_cyclical_data = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\validation_data\Consumer_soc_0.5.csv'

#Technology_envirnomental
validation_tech_data = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\validation_data\Technolo_env_0.5.csv'

#financial services_governance
validation_financials_data = r'C:\Users\Mohammed Al Harmoudi\Documents\GitHub\esg_nlp\data\validation_data\Financia_gov_0.5.csv'


In [8]:
validation_cyc=pd.read_csv(validation_cyclical_data, usecols=[1,2])
validation_tech=pd.read_csv(validation_tech_data, usecols=[1,2])
validation_financials=pd.read_csv(validation_financials_data, usecols=[1,2])

In [9]:
np.any(ticker_library.ticker == "BBWI")

NameError: name 'ticker_library' is not defined

In [10]:
ticker_library = pd.read_csv(os.path.join("data", "tickers.csv"))

def get_cik(ticker):
    """ Get the cik for the ticker specified by the input argument 
    Input:
        ticker(str): ticker of the company e.g. "FB"
    """

    return ticker_library[ticker_library.ticker == ticker].secfilings.values[0][-10:]
    
def ngrams(s, n):
    """ Get all the n-gram for input texts s
    Input:
        s (str): A string of texts with each word separated by a whitespace
        n (int): n-gram to extract
    Return:
        [str]: A list of string in the following format ([['a', 'b'], ['b', 'c'], ['c', 'd']])
    """
    
    s = s.split(' ')
    output = []
    for i in range(len(s) - n + 1):
        output.append(s[i:i+n])

    return output

def get_count(doc, df_dict, n_min, n_max):
    """ Count the number of good and bad words occurred in the document
    Input:
        doc (str): A string with all the words in the documents
        df_dict (pd.DataFrame): A DataFrame with word and isGood column, generated by previous section
        n_min, n_max (int): specify the ngram range used to generate the dictionary, should be consistent with how df_dict is generated
    Return:
        (dict): A dictionary with value good_count and bad_count
    """
    grams = []
    for n in range(n_min, n_max + 1):
        grams.extend([' '.join(li) for li in ngrams(doc, 2)])
    
    good_count = bad_count = 0
    
    for g in grams:
        if g in df_dict["word"].values:
            val = df_dict[df_dict["word"] == g]["isGood"].values
            if val == 1:
                good_count += 1
            elif val == -1:
                bad_count += 1

    return {"good_count": good_count, "bad_count": bad_count}
    
def validation(df_topk, val_tickers, dict_threshold):
    """ Perform the validation step
    The validation rationale: Companies whose score are in upper 50% group are considered "bad" companies and the corresponding val_true = 1; 0 otherwise (in lower 50% group, which is considered a good company)
    Input:
        df_topk (pd.DataFrame): containes the sector specific dict
        val_tickers (list): A list of tickers to be validated
        dict_threshold (int): A gram is considered as a good word if its good_nums - bad_nums > threshold
    """
    diff = df_topk["good_nums"] - df_topk["bad_nums"]
    df_topk["isGood"] = diff.apply(lambda x: 1 if x > dict_threshold else (
        -1 if x < -dict_threshold else 0))

    # 1 if good_nums - bad_nums > threshold; -1 if good_nums - bad_nums < -threshold; 0 otherwise
    val_ciks = [get_cik(ticker) for ticker in val_tickers]
    
    ret_texts = get_texts(val_ciks, val_tickers) 

    val_pred = []
    for doc in tqdm(ret_texts["docs"]):
        ret = get_count(doc, df_topk[["word", "isGood"]], 2, 3)

        if ret["good_count"] - ret["bad_count"] > 0:
            val_pred.append(1)
        else:
            val_pred.append(0)
    
    print("val_pred: {}".format(val_pred))
    
    return val_pred




C:\Users\Mohammed Al Harmoudi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
print(len((validation_financials['good'])))

10


In [12]:
import numpy as np
val_true = np.concatenate([([i]*len(validation_financials['good'])) for i in [1,0]], axis=0).tolist()
val_true

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [13]:
# financials sector, governance
val_tickers_financials = (pd.concat([validation_financials['good'], validation_financials['bad']])).tolist()

# TODO: Update to the actual tickers to test
# val_tickers = ["FB", "GOOG"] # should be a list of tickers you split into test set

val_ciks = [get_cik(ticker) for ticker in val_tickers_financials]

val_pred = validation(df_financials, val_tickers_financials, 2)   # df is the dictionary with good_num, bad_num, diff

# val_true (list): A list of true labels for each companies (1 for good company, 0 for bad)
val_true = np.concatenate([([i]*len(validation_financials['good'])) for i in [1,0]], axis=0).tolist()
# val_true = [1, 1]

cm = confusion_matrix(val_true, val_pred)
prec = cm[1][1]/(cm[1][1]+cm[1][0])
rec = cm[1][1]/(cm[1][1]+cm[0][1])
acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

print("precision: \n{}".format(prec))
print("recall: \n{}".format(rec))
print("accuracy: \n{}".format(acc))
print("Confusion Matrix: \n{}".format(cm))

20it [00:01, 14.56it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [04:37<00:00, 13.88s/it]

val_pred: [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
precision: 
0.7
recall: 
0.875
accuracy: 
0.8
Confusion Matrix: 
[[9 1]
 [3 7]]


In [ ]:
df_financials.isGood.value_counts()

In [26]:
# 1 is numb of good words
# -1 is numb of bad words
df_financials.isGood.value_counts()

 0    106
-1     63
 1     31
Name: isGood, dtype: int64

In [14]:
# technology sector, envirnomental
val_tickers_tech = (pd.concat([validation_tech['good'], validation_tech['bad']])).tolist()

# TODO: Update to the actual tickers to test
# val_tickers = ["FB", "GOOG"] # should be a list of tickers you split into test set

val_ciks = [get_cik(ticker) for ticker in val_tickers_tech]

val_pred = validation(df_tech, val_tickers_tech, 2)   # df is the dictionary with good_num, bad_num, diff

# val_true (list): A list of true labels for each companies (1 for good company, 0 for bad)

val_true = np.concatenate([([i]*len(validation_tech['good'])) for i in [1,0]], axis=0).tolist()
# val_true = [1, 1]
cm = confusion_matrix(val_true, val_pred)
prec = cm[1][1]/(cm[1][1]+cm[1][0])
rec = cm[1][1]/(cm[1][1]+cm[0][1])
acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

print("precision: \n{}".format(prec))
print("recall: \n{}".format(rec))
print("accuracy: \n{}".format(acc))
print("Confusion Matrix: \n{}".format(cm))

20it [00:00, 21.22it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:30<00:00, 10.53s/it]

val_pred: [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1]
precision: 
0.9
recall: 
0.5625
accuracy: 
0.6
Confusion Matrix: 
[[3 7]
 [1 9]]


In [29]:
df_tech.isGood.value_counts()

 0    88
 1    79
-1    33
Name: isGood, dtype: int64

In [8]:
# Consumer cyclical sector, social
val_tickers_cyc = (pd.concat([validation_cyc['good'], validation_cyc['bad']])).tolist()

# TODO: Update to the actual tickers to test
# val_tickers = ["FB", "GOOG"] # should be a list of tickers you split into test set

val_ciks = [get_cik(ticker) for ticker in val_tickers_cyc]

val_pred = validation(df_consumer, val_tickers_cyc, 2)   # df is the dictionary with good_num, bad_num, diff

# val_true (list): A list of true labels for each companies (1 for good company, 0 for bad)
val_true = np.concatenate([([i]*len(validation_cyc['good'])) for i in [1,0]], axis=0).tolist()
# val_true = [1, 1]
cm = confusion_matrix(val_true, val_pred)
prec = cm[1][1]/(cm[1][1]+cm[1][0])
rec = cm[1][1]/(cm[1][1]+cm[0][1])
acc = (cm[1][1] + cm[0][0]) /(cm[1][0]+cm[0][1] + cm[1][1] + cm[0][0])

print("precision: \n{}".format(prec))
print("recall: \n{}".format(rec))
print("accuracy: \n{}".format(acc))
print("Confusion Matrix: \n{}".format(cm))

SEE
BBWI
NKE
TJX
HBI
PKG
BLL
LKQ
HAS
F
AMZN
TSLA
LEN
CCL
HLT
BKNG
EBAY
PVH
SEE
BBWI
NKE
TJX
HBI
PKG
BLL
LKQ
HAS
F
AMZN
TSLA
LEN


0it [00:00, ?it/s]
  0%|           | 0/1 [00:00<?, ?it/s]

CCL
HLT
BKNG
EBAY
PVH
Scraping CIK 0000701985



100%|███| 1/1 [00:06<00:00,  6.81s/it]

  0%|           | 0/1 [00:00<?, ?it/s]

Scraping CIK 0000701985



100%|███| 1/1 [00:09<00:00,  9.12s/it]
18it [00:30,  1.70s/it]
100%|█| 18/18 [03:02<00:00, 10.17s/it]

val_pred: [1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1]
precision: 
0.8888888888888888
recall: 
0.5714285714285714
accuracy: 
0.6111111111111112
Confusion Matrix: 
[[3 6]
 [1 8]]


## Appendix

In [233]:
# To remove all the previous files earlier than 2020
for cik in tqdm(bad_cik):
    tenk_path = os.path.join("data", "10k", cik, "rawtext")
    all_raws = os.listdir(tenk_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenk_path, filename))
    
    tenq_path = os.path.join("data", "10q", cik, "rawtext")
    all_raws = os.listdir(tenq_path)
    for filename in all_raws:
        if filename[0] == '.':
            continue
        year = int(filename.split('_')[1].split('-')[0])
        # print(year)
        if year < 2020:
            # print(os.path.join(tenk_path, filename))
            os.remove(os.path.join(tenq_path, filename))


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 440.37it/s]


In [234]:
df = pd.read_csv("energy_good_vs_bad_uni_bi_tri.csv", index_col=0)

FileNotFoundError: [Errno 2] File energy_good_vs_bad_uni_bi_tri.csv does not exist: 'energy_good_vs_bad_uni_bi_tri.csv'

In [ ]:
df.head()

In [ ]:
comp = pd.read_csv("data/sp500_component_stocks.csv")


In [ ]:
comp[comp['A'] == "C"]["Agilent Technologies Inc."]

In [ ]:

comp[comp['A'].isin(good_companies)]["Agilent Technologies Inc."]

In [ ]:
comp[comp['A'].isin(bad_companies)]["Agilent Technologies Inc."]